# Skyborn Emergent Constraints Demo

This notebook demonstrates the emergent constraint methods and statistical calculations available in the new Skyborn `calc` module. 

Emergent constraints are a powerful method in climate science for reducing uncertainty in future projections by utilizing observable relationships between models and observations.

## Contents
1. **导入必要的库** - Import required libraries
2. **定义并调用函数** - Define and call emergent constraint functions  
3. **使用循环结构** - Apply constraints using loops
4. **异常处理示例** - Handle exceptions in data processing

## 1. 导入必要的库 (Import Required Libraries)

In [1]:
# Import standard libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os

# Add Skyborn to path for development
sys.path.insert(0, os.path.join(os.path.dirname('__file__'), '..', 'src'))

try:
    # Import Skyborn calc functions
    from skyborn.calc import (
        calc_GAUSSIAN_PDF,
        calc_PDF_EC,
        calc_PDF_EC_PRIOR, 
        find_std_from_PDF,
        linear_regression,
        pearson_correlation
    )
    print("✅ Successfully imported Skyborn calc functions!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please ensure Skyborn is properly installed or run from the correct directory.")

✅ Successfully imported Skyborn calc functions!


## 2. 定义并调用函数 (Define and Call Functions)

Let's create some synthetic climate data and demonstrate the emergent constraint functions.

In [2]:
def generate_synthetic_climate_data(n_models=20):
    """
    Generate synthetic climate model data for emergent constraint demonstration.
    
    Parameters:
    -----------
    n_models : int
        Number of climate models to simulate
        
    Returns:
    --------
    tuple : (x_data, y_data)
        Constraint variable and target variable data
    """
    np.random.seed(42)  # For reproducible results
    
    # Generate constraint variable (e.g., model sensitivity parameter)
    x_models = np.random.normal(3.0, 0.8, n_models)
    
    # Generate target variable with correlation to constraint
    # This represents the emergent relationship
    noise = np.random.normal(0, 0.3, n_models)
    y_models = 1.5 * x_models + 2.0 + noise
    
    return x_models, y_models

# Generate synthetic data
x_constraint, y_target = generate_synthetic_climate_data()

print(f"Generated data for {len(x_constraint)} climate models")
print(f"Constraint variable range: {x_constraint.min():.2f} to {x_constraint.max():.2f}")
print(f"Target variable range: {y_target.min():.2f} to {y_target.max():.2f}")

# Calculate correlation
correlation = pearson_correlation(x_constraint, y_target) 
print(f"Pearson correlation: {correlation:.3f}")

Generated data for 20 climate models
Constraint variable range: 1.47 to 4.26
Target variable range: 3.89 to 8.05
Pearson correlation: 0.968


## 3. 使用循环结构 (Apply Constraints Using Loops)

Now we'll use loops to apply the emergent constraint method across different scenarios.

In [3]:
# Create observation constraint (simulated observational data)
obs_mean = 2.8  # Observed constraint value
obs_std = 0.2   # Observational uncertainty

# Create grid for PDF calculations
x_grid = np.linspace(1.0, 5.0, 100)
y_grid = np.linspace(4.0, 12.0, 100)

# Calculate observational PDF
obs_pdf = calc_GAUSSIAN_PDF(obs_mean, obs_std, x_grid)

print("Applying emergent constraints to multiple scenarios...")

# Loop through different observational uncertainties
scenarios = [0.1, 0.2, 0.3, 0.4]
results = []

for i, uncertainty in enumerate(scenarios):
    print(f"Scenario {i+1}: Observational uncertainty = {uncertainty}")
    
    # Calculate constraint PDF for this scenario
    obs_pdf_scenario = calc_GAUSSIAN_PDF(obs_mean, uncertainty, x_grid)
    
    # Convert to xarray for compatibility
    tmp_x = xr.DataArray(x_constraint, dims=['model'])
    tmp_y = xr.DataArray(y_target, dims=['model'])
    
    # Apply emergent constraint
    try:
        constrained_pdf, sigma_y, mean_y = calc_PDF_EC(tmp_x, tmp_y, x_grid, y_grid, obs_pdf_scenario)
        
        results.append({
            'scenario': i+1,
            'uncertainty': uncertainty,
            'constrained_mean': mean_y,
            'constrained_std': sigma_y,
            'pdf': constrained_pdf
        })
        
        print(f"   Constrained mean: {mean_y:.2f}")
        print(f"   Constrained std:  {sigma_y:.2f}")
        
    except Exception as e:
        print(f"   Error in scenario {i+1}: {e}")

print(f"\nProcessed {len(results)} scenarios successfully")

Applying emergent constraints to multiple scenarios...
Scenario 1: Observational uncertainty = 0.1
   Constrained mean: 6.10
   Constrained std:  0.24
Scenario 2: Observational uncertainty = 0.2
   Constrained mean: 6.10
   Constrained std:  0.32
Scenario 3: Observational uncertainty = 0.3
   Constrained mean: 6.10
   Constrained std:  0.40
Scenario 4: Observational uncertainty = 0.4
   Constrained mean: 6.10
   Constrained std:  0.57

Processed 4 scenarios successfully
   Constrained mean: 6.10
   Constrained std:  0.40
Scenario 4: Observational uncertainty = 0.4
   Constrained mean: 6.10
   Constrained std:  0.57

Processed 4 scenarios successfully


## 4. 异常处理示例 (Exception Handling Examples)

Robust data processing requires proper exception handling, especially when working with climate data.

In [4]:
def robust_emergent_constraint(x_data, y_data, obs_mean, obs_std):
    """
    Apply emergent constraint with robust error handling.
    """
    try:
        # Validate input data
        if len(x_data) != len(y_data):
            raise ValueError("X and Y data must have the same length")
        
        if len(x_data) < 3:
            raise ValueError("Need at least 3 data points for meaningful constraint")
        
        if obs_std <= 0:
            raise ValueError("Observational uncertainty must be positive")
        
        # Check for invalid values
        if np.any(np.isnan(x_data)) or np.any(np.isnan(y_data)):
            raise ValueError("Data contains NaN values")
        
        # Apply constraint
        x_grid = np.linspace(x_data.min() - 1, x_data.max() + 1, 50)
        y_grid = np.linspace(y_data.min() - 2, y_data.max() + 2, 50)
        
        obs_pdf = calc_GAUSSIAN_PDF(obs_mean, obs_std, x_grid)
        
        tmp_x = xr.DataArray(x_data, dims=['model'])
        tmp_y = xr.DataArray(y_data, dims=['model'])
        
        constrained_pdf, sigma_y, mean_y = calc_PDF_EC(tmp_x, tmp_y, x_grid, y_grid, obs_pdf)
        
        return {
            'success': True,
            'mean': mean_y,
            'std': sigma_y,
            'pdf': constrained_pdf,
            'message': 'Constraint applied successfully'
        }
        
    except ValueError as e:
        return {
            'success': False,
            'error': f"Value Error: {e}",
            'message': 'Check input data validity'
        }
    except Exception as e:
        return {
            'success': False,
            'error': f"Unexpected Error: {e}",
            'message': 'Contact developer for support'
        }

# Test with various problematic scenarios
test_cases = [
    {
        'name': 'Valid data',
        'x': x_constraint,
        'y': y_target,
        'obs_mean': 2.8,
        'obs_std': 0.2
    },
    {
        'name': 'Mismatched lengths',
        'x': x_constraint[:5],
        'y': y_target,
        'obs_mean': 2.8,
        'obs_std': 0.2
    },
    {
        'name': 'Too few points',
        'x': x_constraint[:2],
        'y': y_target[:2],
        'obs_mean': 2.8,
        'obs_std': 0.2
    },
    {
        'name': 'Negative uncertainty',
        'x': x_constraint,
        'y': y_target,
        'obs_mean': 2.8,
        'obs_std': -0.1
    }
]

print("Testing exception handling:")
print("=" * 50)

for test in test_cases:
    print(f"\nTest: {test['name']}")
    result = robust_emergent_constraint(test['x'], test['y'], test['obs_mean'], test['obs_std'])
    
    if result['success']:
        print(f"✅ Success: Mean = {result['mean']:.2f}, Std = {result['std']:.2f}")
    else:
        print(f"❌ Failed: {result['error']}")
        print(f"   Advice: {result['message']}")

print("\n🎯 Exception handling demonstration completed!")

Testing exception handling:

Test: Valid data
✅ Success: Mean = 6.05, Std = 0.17

Test: Mismatched lengths
❌ Failed: Value Error: X and Y data must have the same length
   Advice: Check input data validity

Test: Too few points
❌ Failed: Value Error: Need at least 3 data points for meaningful constraint
   Advice: Check input data validity

Test: Negative uncertainty
❌ Failed: Value Error: Observational uncertainty must be positive
   Advice: Check input data validity

🎯 Exception handling demonstration completed!


## Summary

This notebook demonstrated the key features of the Skyborn `calc` module:

1. **Library Management**: Proper import and error handling for dependencies
2. **Function Definition**: Creating and calling emergent constraint functions  
3. **Loop Processing**: Applying constraints across multiple scenarios
4. **Exception Handling**: Robust error management for data processing

The emergent constraint method is a powerful tool for reducing uncertainty in climate projections by leveraging observable relationships between models and real-world data.

### Next Steps

- Explore the DATA folder with real climate model outputs
- Try different constraint variables and targets
- Implement custom PDF functions for specific applications
- Combine with the other Skyborn modules for complete analysis workflows